In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import class_weight
import numpy as np


Retina images have been taken from kaggle
https://www.kaggle.com/c/aptos2019-blindness-detection/data?select=train_images

In [67]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/14774/875431/compressed/train_images.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1633975983&Signature=jxJ9u5saJ0msjVnG3CmJmiSvkdTD5J3JQy%2FYy6TFN9jlR53gD1zETwa97M7wQnMiaKpdVjyM4156wivFz0D0%2BX9LEAkVZm2LGn1RsrpwfsuknooPSdncf5rqWQvDIwON%2F4Y7fhjgfg1WC%2BwtOa6RUcGJ23M%2Blpp45voz%2Fd%2FF0VVlULKdc9E7vsfrbcfrY%2FhRgHe2Xqs5lGwvvNoANvmZsB1WOAX%2FDU6dXheY7NDxaTCWZYQFcd6GYLFv1GswLiqUbPLlIAheLJH2dbqs%2BrnsQOjCpzADawZtAx3m0HqboJ4kkdBqoIDt2tBBLpjG99WOoyP0G3sNKd8i2WCdDAYl%2Fg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain_images.zip" -c -O 'train_images.zip'

--2021-10-08 18:13:27--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/14774/875431/compressed/train_images.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1633975983&Signature=jxJ9u5saJ0msjVnG3CmJmiSvkdTD5J3JQy%2FYy6TFN9jlR53gD1zETwa97M7wQnMiaKpdVjyM4156wivFz0D0%2BX9LEAkVZm2LGn1RsrpwfsuknooPSdncf5rqWQvDIwON%2F4Y7fhjgfg1WC%2BwtOa6RUcGJ23M%2Blpp45voz%2Fd%2FF0VVlULKdc9E7vsfrbcfrY%2FhRgHe2Xqs5lGwvvNoANvmZsB1WOAX%2FDU6dXheY7NDxaTCWZYQFcd6GYLFv1GswLiqUbPLlIAheLJH2dbqs%2BrnsQOjCpzADawZtAx3m0HqboJ4kkdBqoIDt2tBBLpjG99WOoyP0G3sNKd8i2WCdDAYl%2Fg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain_images.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.214.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8602291542 (8.0G) [application/zip]
Saving to: ‘train_images.zip’



In [68]:
import shutil
shutil.unpack_archive('train_images.zip','data')

In [69]:
#loading our data
df=pd.read_csv("train.csv")
# creating a column with extension of .png , as id_code column contain only id
df['add']=df['id_code']+'.png'
# creating a column which contains the path of images
df['path']='data/'+df['add']

In [70]:
df.head()

,id_code,diagnosis,add,path
0,000c1434d8d7,2,000c1434d8d7.png,data/000c1434d8d7.png
1,001639a390f0,4,001639a390f0.png,data/001639a390f0.png
2,0024cdab0c1e,1,0024cdab0c1e.png,data/0024cdab0c1e.png
3,002c21358ce6,0,002c21358ce6.png,data/002c21358ce6.png
4,005b95c28852,0,005b95c28852.png,data/005b95c28852.png


In [4]:
#ref : - https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy

def crop(img,tol=7):
 # here tol is tolerance  
  '''
  this crop function is used for removing darkparts arounds the image
  '''

  if img.ndim==2:
# this loop is used for cropping GRAY images
    mask=img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]
  elif img.ndim==3:
# this loop is used for cropping color images    
    grayimg=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    mask=grayimg>tol
    shap=img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
    if shap==0:
# image is too dark so that we crop out everything   
      return img
    else:
      img0=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
      img1=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
      img2=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
      img=np.stack([img0,img1,img2],axis=-1)
    return img 

def circlecrop(img):
  '''
  used for cropping image in a circular way from image centre
  '''

  h,w,d= img.shape
  x = int(w/2)
  y = int(h/2)
  r = np.amin((x,y))
  circle_img = np.zeros((h,w), np.uint8)
  cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
  img = cv2.bitwise_and(img, img, mask=circle_img) 
  return img


def ben(img,sigmaX=10):
  '''
  Ben Graham's method to improve lighting condition.

  '''
  image=cv2.addWeighted( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
  return image


In [84]:
from keras.models import load_model
from sklearn.metrics import cohen_kappa_score
from keras_preprocessing.image import ImageDataGenerator

def kappa(y_true, y_pred):
  y_true = np.argmax(y_true, axis=-1)
  y_pred = np.argmax(y_pred, axis=-1)
  kappa_score = cohen_kappa_score(y_true, y_pred, weights='quadratic')
  return kappa_score

def kappa_metric(y_true, y_pred):
  kappa_score = tf.py_function(func=kappa, inp=[y_true, y_pred], Tout=tf.float32)
  return kappa_score

In [9]:
#loading our best model
model=load_model('modeld.h5',custom_objects={"kappa_metric":kappa_metric})

In [71]:
X=df['path']
Y=df['diagnosis']

In [73]:
def func1(X):
  img = cv2.imread(X)
  img=crop(img)
  img = cv2.resize(img,(256,256),interpolation=cv2.INTER_AREA)
  img=circlecrop(img)
  img=ben(img)
  img = np.reshape(img,[1,256,256,3])
  cd = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                          rotation_range=360,brightness_range=[0.5, 1],
                           zoom_range = 0.2,rescale=1./255)
  cg = cd.flow(img,batch_size=1)
  tp = model.predict(cg)

  op=np.argmax(tp)
  return op


In [74]:
def func2(X,Y):
  op=func1(X)
  print("Actual categorized stage is ",Y)
  print("Predicted stage is",op)

In [75]:
func2(X[0],Y[0])

Actual categorized stage is  2
Predicted stage is 2


In [76]:
func2(X[1],Y[1])

Actual categorized stage is  4
Predicted stage is 4


In [80]:
func2(X[2890],Y[2890])

Actual categorized stage is  4
Predicted stage is 2


In [78]:
func2(X[36],Y[36])

Actual categorized stage is  0
Predicted stage is 0


In [79]:
func2(X[49],Y[49])

Actual categorized stage is  3
Predicted stage is 4


In [82]:
func2(X[569],Y[569])

Actual categorized stage is  0
Predicted stage is 0


In [83]:
func2(X[71],Y[71])

Actual categorized stage is  1
Predicted stage is 2
